### import 및 전역변수
- 컷오프일 parquet 폴더의 파일 읽기 : 채무자, 계좌, 보증인, 개회, 신복
- 매각일 추가 파일 다운 및 읽기 : 매각일기준계좌, 메모, 법조치, 입금
- ★ 매각일 계좌정보는 굳이 줄 필요 없겠다. (혹시 달라고 할 수도 있으니 코드 수정은 안 함, 출력물에서 삭제하자.)
- ★ 컷오프 연체이자, 현재합계 출력물에서 raw_data보고 수정할 것

In [2]:
import pandas as pd
from tqdm import tqdm
import re, os, functions
from os.path import join

account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
memo_dtype = {'채무자키' : str, '메모키':str}
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키' : str, '입금키':str, '계좌키':str, '입금계좌번호':str, '계좌번호':str, '타입금키':str, '타채무자키':str, '보증인키':str}

In [5]:
################################################################
company = "솔림"
매각사 = "신안어소시"

cut_date = "250824"
sell_date = "250912"

cut_ext = ".parquet" # 자산정리하면서 parquet 만들었으니
sell_ext = ".parquet" # 매각일 전산자료. 처음 받았다면 xlsx  ###############

path_save_base = r"D:\3.자산\POOL별 관리자산\솔림\POOL 8 번자료 (, JB우리캐)\250630\신용회복_개인_매각\Layout\전산 rawdata\대체건" # 매각관리 폴더
path_sell = join(path_save_base, "전산 rawdata") # 매각 폴더 안에 전산로우데이터 다운, 메모-계좌-법조치-입금
path_매각키리스트 = join(path_save_base, "신안어소시-02_매각키리스트"+".xlsx") # 채무자키, 계좌키 만
################################################################

path_sell_parquet = join(path_sell, "parquet")
path_cut_parquet = join(r"D:\3.자산\전산 dataset", company, cut_date, "parquet")
fullfullpath_rst = join(path_save_base, "솔림_전산자료_"+매각사+"_"+cut_date+".xlsx")

cut_date_to_input = "20"+cut_date[:2]+"-"+cut_date[2:4]+"-"+cut_date[4:]
sell_date_to_input = "20"+sell_date[:2]+"-"+sell_date[2:4]+"-"+sell_date[4:]

### 파일 읽기

In [6]:
# 파일읽기
cut_파일검색어 = ["채무자조회새창_", "계좌조회새창_", "보증인새창_", r"개인회생새창_.*기준데이터", "신용회복_기준데이터"]  # 신복, 개회 기준데이터로 그래야 자산정리와 같고, 날짜랑도 맞는다.
sell_파일검색어 = ["계좌", "메모", "법조치", "입금"] # 법조치 : 새채무자키가 같은 채무자키로 검색해야.

cut_files = []        
for kw in cut_파일검색어 : 
    if re.search("보증인새창", kw) : # 새보증인상태 파일 말고 원 파일 읽자
        cut_files.append(functions.키워드로파일명찾기(path_cut_parquet, kw,'새보증인상태',전체경로=True))
    else : 
        cut_files.append(functions.키워드로파일명찾기(path_cut_parquet, kw,전체경로=True))

sell_files = []
for kw in sell_파일검색어 : 
    if sell_ext == ".xlsx" :
        path = path_sell
    else :
        path = path_sell_parquet
    sell_files.append(functions.키워드로파일명찾기(path, kw,전체경로=True))
    

# 자산확정일 기준 자료들
debt = pd.read_parquet(join(path_cut_parquet, cut_files[0]))
cut_accout = pd.read_parquet(join(path_cut_parquet, cut_files[1]))
grt = pd.read_parquet(join(path_cut_parquet, cut_files[2]))
rehabilitation = pd.read_parquet(join(path_cut_parquet, cut_files[3]))
credit = pd.read_parquet(join(path_cut_parquet, cut_files[4]))

# 매각일 기준 자료들
if sell_ext == ".xlsx" : 
    sell_account = pd.read_excel(sell_files[0], dtype=account_dtype ).fillna("")
    memo = pd.read_excel(sell_files[1], dtype=memo_dtype ).fillna("")
    event = pd.read_excel(sell_files[2], dtype=event_dtype ).fillna("")
    deposit = pd.read_excel(sell_files[3], dtype=deposit_dtype ).fillna("")
else : # parquet
    sell_account = pd.read_parquet(sell_files[0])
    memo = pd.read_parquet(sell_files[1])
    event = pd.read_parquet(sell_files[2])
    deposit = pd.read_parquet(sell_files[3])

# 신복기준데이터의 No.열 없애기
credit = credit.drop(['순번'], axis=1) # No. > 순번

# 매각리스트
keys = pd.read_excel(path_매각키리스트, dtype=str).fillna("")

In [ ]:
# # 매각일 자료 parquet 저장 -----------
# if not os.path.exists(path_sell_parquet) : 
#     os.mkdir(path_sell_parquet)
    
# functions.to_parquet(sell_account, join(path_sell_parquet, os.path.splitext(os.path.dirname(sell_files[0]))[0]+".parquet"))
# functions.to_parquet(memo, join(path_sell_parquet, os.path.splitext(os.path.dirname(sell_files[1]))[0]+".parquet"))
# functions.to_parquet(event, join(path_sell_parquet, os.path.splitext(os.path.dirname(sell_files[2]))[0]+".parquet"))
# functions.to_parquet(deposit, join(path_sell_parquet, os.path.splitext(os.path.dirname(sell_files[3]))[0]+".parquet"))

### 데이터 추출

In [7]:
# merge(df-df)를 위해 키 중복값 제거
debtkeys = keys[["채무자키"]].drop_duplicates('채무자키') # df리턴
accountkeys = keys[["계좌키"]].drop_duplicates('계좌키') # df리턴 # 중복값 없겠지만 혹시 모르니
# 채무자키와 새채무자키
new_debtkeys = debt[["채무자키","관리자비고"]].merge(debtkeys, how="right").rename(columns = {"관리자비고":"새채무자키"})
print(f'채무자키:{len(debtkeys)}, 계좌키:{len(accountkeys)}, 새채무자키:{len(new_debtkeys)}')

채무자키:2, 계좌키:2, 새채무자키:2


In [8]:
# 법조치 새채무자키 추가하기 (new_debekeys의 채무자키에 중복값이 없으므로 숫자변화 없다)
print(len(event))
event = event.merge(new_debtkeys, on='채무자키', how='left')
print(len(event))

1
1


In [9]:
# 추출하기 : 법조치는 새채무자키로 추출
# 자산확정일 : 채무자, 계좌, 보증인, 개인회생, 신용회복
# 매각일 : 계좌, 입금(계좌조회,계좌별표시,기간제한x), 메모, 법조치(새채무자키)
# 채무자 : 채무자키
debt = debt.merge(debtkeys, how='inner')
# 계좌_확정일 : 계좌키 - 계좌키 누락/일부 매각 없어야
cut_accout = cut_accout.merge(accountkeys, how='inner')
# 보증인 : 계좌키
grt = grt.merge(accountkeys, how='inner') # 보증인
# 개인회생 : 계좌키
rehabilitation = rehabilitation.merge(accountkeys, how='inner')
# 신용회복 : 계좌키
credit = credit.merge(accountkeys, how='inner')
# 법조치, 채무자키 기준 vs 새채무자키 기준 - 계좌키는 제대로 등록이 되어있지 않으므로 사용할 수 없다.
event_채무자키 = event[event.채무자키.isin(keys.채무자키)].copy()

print(f"법조치 - 채무자키로 법조치 추출 : {len(event_채무자키)}, 새채무자키로 법조치 추출 : {len(event)}")
# 메모 : 채무자키
memo = memo.merge(debtkeys, how='inner')
# 계좌_매각일
sell_account = sell_account.merge(accountkeys, how='inner')
# 입금 : 계좌키
deposit = deposit.merge(accountkeys, how='inner')

# 법조치 새채무자키든 채무자키든 결과차이 없다면 새채무자키 열 삭제 
if len(event_채무자키) == len(event) : 
    event.drop(['새채무자키'], axis=1)

법조치 - 채무자키로 법조치 추출 : 1, 새채무자키로 법조치 추출 : 1


### 칼럼 편집

In [10]:
dfs = [debt, cut_accout, grt, rehabilitation, credit, event, memo, deposit, sell_account]
df_names = ["채무자", "계좌", "보증인", "개인회생", "신용회복", "법조치_"+sell_date, "memo_"+sell_date, "deposit_"+sell_date, "계좌_"+sell_date]

# 문제되는 칼럼 살피기
x = set()
y = {name: [] for name in df_names}
for i, df in enumerate(dfs):
    for col in df.columns :
        if re.search('이자기준일', col) :
            y.get(df_names[i]).append(col)
            x.add(col)
print(x)
for k,v in y.items() : 
    print(k, ":", v)

{'최초이자기준일'}
채무자 : []
계좌 : ['최초이자기준일']
보증인 : []
개인회생 : []
신용회복 : []
법조치_250912 : []
memo_250912 : []
deposit_250912 : ['최초이자기준일']
계좌_250912 : ['최초이자기준일']


In [11]:
# 컬럼 내용 편집
# 초기화할 열 목록
columns_to_reset = ['담당자','담당자키','담당핸드폰','송무담당', '담당입금구분', '담당문자전송', '담당연락처', '전담당자', '담당변경일', '지점','지역지점', "채무상태", 
     '부서', '팀', '등록파일', '등록유저', '수정파일', '법조치파일','등록유저', '수정유저','OPB', '담보제공처','리파이낸싱', '담보제공구분', '채권매각일', '채권매각대금', '채권매각처', '채권환매대금','채권환매일',
     '채권손실일', '채권손실금액', '계약일', '입금은행', '상대방입금은행', '입금계좌번호', '입금순서','요청자','요청일','승인자','승인일','전결자','전결일'] # OPB 추가:우리 매입당시 OPB 알려줄 필요없으니 알아서 계산해서 쓰라고
columns_to_input = ["매입일", "컷오프일", "최초이자기준일"]

# 모든 데이터프레임에 대해 열 초기화
for df in dfs:
    for col in columns_to_reset: # 삭제
        if col in df.columns:
            df[col] = ""
    for ind, col in enumerate(columns_to_input): # 컷오프일 or 매입일 작성
        if col in df.columns:
            if ind==0 : df[col] = sell_date_to_input
            else : df[col] = cut_date_to_input

In [12]:
# 메모시트 등록일자 편집 및 매각메모 삭제
############################################################
삭제할메모내용 = '매각 / ㈜신안어소시에이츠대부 / 02-6949-0741\n컷오프일 : 2025-06-30, 계약일 2025-07-28, 거래종료일 2025-08-28'
############################################################
# 공지 기호 삭제
memo["등록일자"] = memo.등록일자.str.replace("★ | ★", "", regex=True)
# 매각 메모 삭제
before = len(memo)
memo = memo[~(memo.메모내용 == 삭제할메모내용)]
memo.reset_index(drop=True)
after = len(memo)
print(f"삭제한 메모수 : {before} - {after} = {before-after}, 매각한 채무자키 수 : {len(debtkeys)}")
# 등록일자순 정렬
memo = memo.sort_values(['등록일자'], ascending=False)
# 메모색상, 공지여부
memo['메모색상'] = "검정"
memo['공지여부'] = "N"

삭제한 메모수 : 39 - 39 = 0, 매각한 채무자키 수 : 2


### 저장

In [13]:
# 엑셀 저장 (파일이 오류나네.. 근데 내용은 문제 없어서, 복구 후 다른이름으로 저장하면 괜찮았다.)
# dfs에 변경된 내용 반영 (drop과 sort가 반영이 안 되네..)
dfs = [debt, cut_accout, grt, rehabilitation, credit, event, memo, deposit, sell_account]
df_names = ["채무자", "계좌", "보증인", "개인회생", "신용회복", "법조치_"+sell_date, "메모_"+sell_date, "입금_"+sell_date, "계좌_"+sell_date]
dict_sheet_df = dict(zip(df_names, dfs))

with pd.ExcelWriter(fullfullpath_rst, engine='openpyxl', mode = 'w') as writer:
    for sheetname, df_name in dict_sheet_df.items():
        df_name.to_excel(writer, sheet_name = sheetname, index=False)

In [ ]:
# readme - 다음을 복사
readme = """
시트명에 날짜가 없는 것은 컷오프일 기준의 전산데이터입니다.
시트명에 날짜가 붙은 것은 매각일(거래종료일) 기준의 전산데이터입니다.

삭제정보 : 불필요한 정보(솔림내부정보 : 담당자, 입금계좌 등), 재작성 해야하는 정보(상태,OPB 등)는 삭제하였습니다.
수정정보 : 매입일, 컷오프일, 최초이자기준일은 금번 계약 내용으로 수정하여 두었습니다.
유지정보(등록시 수정필요) : 매각사, 각종 키값 및 타키값(타채무자키 등)은 참고가 필요할 수 있어 일단 솔림 전산값 그대로 두었으니 전산 등록시 수정하시기 바랍니다.

면책된 보증인 정보는 제거하였습니다.
"""
#    법조치의 새채무자키 - 매각대상에 없는 채무자키가 들어있을 수 있습니다. 이는 채무자키는 다르지만, ....